<a href="https://colab.research.google.com/github/andivaanggoro/andivaanggoromachinelearning/blob/main/NIN_Andiva_Kasih_Anggoro_Putra_1103204031.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Nama : Andiva Kasih Anggoro Putra

NIM : 1103204031

In [1]:
!pip install d2l

In [2]:
import torch
# library utama untuk komputasi tensor dan pembelajaran mendalam (deep learning) menggunakan PyTorch. Library ini menyediakan berbagai fungsi untuk operasi matematika dan manipulasi tensor.
from torch import nn
# digunakan untuk membangun dan melatih jaringan saraf, seperti lapisan jaringan saraf (layers), fungsi aktivasi, dan fungsi kerugian (loss functions).
import d2l.torch as d2l
# menyediakan berbagai fungsi dan utilitas yang membantu dalam pembelajaran mendalam menggunakan PyTorch. Modul ini mencakup fungsi untuk visualisasi, pemrosesan data, dan banyak lagi.



In [3]:
def nin_block(out_channels, kernel_size, strides, padding):
  # nin_block yang mengambil empat argumen
    return nn.Sequential(
        # untuk mengurutkan beberapa lapisan jaringan secara sekuensial.
        nn.LazyConv2d(out_channels, kernel_size, strides, padding), nn.ReLU(),
        # Lapisan konvolusi 2D yang menggunakan jumlah channel output , berarti lapisan ini menunda inisialisasi parameter hingga input yang sebenarnya diberikan.
        nn.LazyConv2d(out_channels, kernel_size=1), nn.ReLU(),
        # Lapisan konvolusi 2D dengan kernel 1x1, yang mengubah jumlah channel tanpa mengubah dimensi spasial. Ini diikuti oleh fungsi aktivasi nn.ReLU().
        nn.LazyConv2d(out_channels, kernel_size=1), nn.ReLU())
    # Lapisan ini mengubah jumlah channel tanpa mengubah dimensi spasial, menambahkan lebih banyak non-linearitas setelahnya.


In [4]:
class NiN(d2l.Classifier):
  # untuk klasifikasi dengan berbagai metode pembelajaran mendalam.
    def __init__(self, lr=0.1, num_classes=10):
      # mendefinisikan dua parameter: lr (learning rate) dan num_classes (jumlah kelas untuk klasifikasi), dengan nilai default masing-masing 0.1 dan 10.
        super().__init__()
        # untuk menginisialisasi objek induk. Ini memastikan bahwa kelas dasar diinisialisasi dengan benar.
        self.save_hyperparameters()
        # menyimpan hiperparameter seperti lr dan num_classes
        self.net = nn.Sequential(
            # input akan melewati setiap lapisan dalam urutan yang diberikan.
            nin_block(96, kernel_size=11, strides=4, padding=0),
            #  merupakan fungsi yang mendefinisikan beberapa lapisan konvolusi dan aktivasi.
            nn.MaxPool2d(3, stride=2),
            # Lapisan pooling maksimum dengan ukuran kernel 3 dan stride 2 untuk mengurangi dimensi spasial.
            nin_block(256, kernel_size=5, strides=1, padding=2),
            # Blok NiN kedua dengan 256 filter, kernel size 5, stride 1, dan padding 2.
            nn.MaxPool2d(3, stride=2),
            # Lapisan pooling maksimum kedua.
            nin_block(384, kernel_size=3, strides=1, padding=1),
            # Blok NiN ketiga dengan 384 filter, kernel size 3, stride 1, dan padding 1.
            nn.MaxPool2d(3, stride=2),
            # Lapisan pooling maksimum ketiga.
            nn.Dropout(0.5),
            # Dropout layer dengan probabilitas dropout 0.5 untuk mengurangi overfitting.
            nin_block(num_classes, kernel_size=3, strides=1, padding=1),
            # Blok NiN terakhir dengan jumlah filter sesuai jumlah kelas, kernel size 3, stride 1, dan padding 1.
            nn.AdaptiveAvgPool2d((1, 1)),
            # Lapisan pooling rata-rata adaptif yang mengubah dimensi output spasial menjadi (1, 1) untuk setiap filter.
            nn.Flatten())
        # Lapisan untuk meratakan output menjadi vektor satu dimensi.
        self.net.apply(d2l.init_cnn)
        # untuk menginisialisasi bobot jaringan saraf dengan cara tertentu, seperti inisialisasi He atau Xavier.


In [5]:
NiN().layer_summary((1, 1, 224, 224))


/usr/local/lib/python3.10/dist-packages/torch/nn/modules/lazy.py:181: UserWarning: Lazy modules are a new feature under heavy development so changes to the API or functionality can happen at any moment.
  warnings.warn('Lazy modules are a new feature under heavy development '


Sequential output shape:	 torch.Size([1, 96, 54, 54])
MaxPool2d output shape:	 torch.Size([1, 96, 26, 26])
Sequential output shape:	 torch.Size([1, 256, 26, 26])
MaxPool2d output shape:	 torch.Size([1, 256, 12, 12])
Sequential output shape:	 torch.Size([1, 384, 12, 12])
MaxPool2d output shape:	 torch.Size([1, 384, 5, 5])
Dropout output shape:	 torch.Size([1, 384, 5, 5])
Sequential output shape:	 torch.Size([1, 10, 5, 5])
AdaptiveAvgPool2d output shape:	 torch.Size([1, 10, 1, 1])
Flatten output shape:	 torch.Size([1, 10])


In [ ]:
model = NiN(lr=0.05)
#  jenis arsitektur jaringan saraf dalam (deep neural network) dan menentukan seberapa besar model diperbarui setiap kali mengalami propagasi balik.
trainer = d2l.Trainer(max_epochs=10, num_gpus=1)
# diinisialisasi dengan parameter max_epochs=10, yang berarti model akan dilatih selama 10 epoch dan menunjukkan bahwa pelatihan akan dilakukan menggunakan satu GPU, jika tersedia.
data = d2l.FashionMNIST(batch_size=128, resize=(224, 224))
# untuk memuat dataset Fashion-MNIST dan diinisialisasi dengan batch_size=128, yang berarti data akan diproses dalam batch berukuran 128. resize=(224, 224) berarti gambar dalam dataset akan diubah ukurannya menjadi 224x224 piksel.
model.apply_init([next(iter(data.get_dataloader(True)))[0]], d2l.init_cnn)
# digunakan untuk menginisialisasi parameter model dan inisialisasi khusus untuk CNN
trainer.fit(model, data)
# digunakan untuk melatih model dengan data yang diberikan.


100%|██████████| 26421880/26421880 [00:01<00:00, 17609895.51it/s]


Extracting ../data/FashionMNIST/raw/train-images-idx3-ubyte.gz to ../data/FashionMNIST/raw



100%|██████████| 29515/29515 [00:00<00:00, 300028.80it/s]


Extracting ../data/FashionMNIST/raw/train-labels-idx1-ubyte.gz to ../data/FashionMNIST/raw



100%|██████████| 4422102/4422102 [00:00<00:00, 5581492.52it/s]


Extracting ../data/FashionMNIST/raw/t10k-images-idx3-ubyte.gz to ../data/FashionMNIST/raw



100%|██████████| 5148/5148 [00:00<00:00, 4935377.60it/s]

Extracting ../data/FashionMNIST/raw/t10k-labels-idx1-ubyte.gz to ../data/FashionMNIST/raw




/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:558: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(
